### Using an OpenAI API key 

In [28]:
from langchain.llms import OpenAI

In [29]:
from dotenv import load_dotenv
import os

load_dotenv()

openai_api_key_env = os.environ.get("OPENAI_API_KEY")

In [30]:
openai_llm = OpenAI(openai_api_key=openai_api_key_env,model_name="gpt-3.5-turbo-instruct",temperature=0.6)

In [31]:
text = "Give a description of an apple in 25 words"

print(openai_llm.predict(text))



An apple is a round, juicy fruit with a shiny, smooth skin that comes in various shades of red, green, and yellow. It has a crisp texture and a sweet and slightly tart flavor.


### Using a HuggingFace access token to use Open Source LLM Model

In [32]:
load_dotenv()
hf_access_token_env = os.environ.get("HUGGINGFACE_ACCESS_TOKEN")

In [33]:
from langchain import HuggingFaceHub
hf_llm = HuggingFaceHub(huggingfacehub_api_token = hf_access_token_env,
                        repo_id="google/flan-t5-large",
                        model_kwargs={"temperature" : 0,"max_length":64})

In [34]:
output = hf_llm.predict("What color is an apple ?")
output

'green'

### Prompt templates & LLMChain

In [35]:
from langchain.prompts import PromptTemplate
promptTemplate = PromptTemplate(input_variables=['country'],
                                template= "Tell me the currency of {country}")

promptTemplate.format(country="Italy")

'Tell me the currency of Italy'

In [36]:
from langchain.chains import LLMChain
chain = LLMChain(llm = openai_llm,
                 prompt = promptTemplate)
chain.run("New Zeland")

'\n\nThe currency of New Zealand is the New Zealand dollar (NZD).'

### Combining multiple chains using Simple Sequential Chain

In [37]:
captial_template = PromptTemplate(input_variables=['Country'],
                                template="Tell me the capital of the {Country}")

capital_chain = LLMChain(llm=openai_llm,prompt=captial_template)

famous_template = PromptTemplate(input_variables=['Capital'],
                                 template="suggest me 3 places to visit in {Capital}")

famous_chain = LLMChain(llm=openai_llm, prompt=famous_template)

In [38]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains=[capital_chain,famous_chain])
print(chain.run("Greece"))



1. Acropolis of Athens - This ancient citadel is home to the iconic Parthenon temple and offers stunning views of the city.

2. Plaka Neighborhood - This charming area is known for its picturesque streets, traditional tavernas, and souvenir shops. It's a great place to experience the local culture and cuisine.

3. National Archaeological Museum - Considered one of the greatest museums in the world, this museum houses an extensive collection of artifacts from ancient Greece, including sculptures, pottery, and jewelry. 


### SequentialChain

In [50]:
from langchain.chains import SequentialChain

capital_template=PromptTemplate(input_variables=['country'],
                                template="Please tell me the capital of the {country}")

capital_chain=LLMChain(llm=openai_llm,prompt=capital_template,output_key="capital")

In [51]:
currency_template = PromptTemplate(input_variables=['country'],
                                template="tell me the currency of {country}")

currency_chain =LLMChain(llm=openai_llm,prompt=currency_template,output_key="currency")

In [52]:

famous_template=PromptTemplate(input_variables=['capital'],
                               template="Suggest me some amazing places to visit in {capital}")

famous_chain=LLMChain(llm=openai_llm,prompt=famous_template,output_key="places")

In [53]:
from langchain.chains import SequentialChain
chain=SequentialChain(chains=[capital_chain,currency_chain,famous_chain],
                      input_variables=['country'],
                      output_variables=['capital','currency',"places"])

In [54]:
chain({'country':"Japan"})


{'country': 'Japan',
 'capital': '\n\nThe capital of Japan is Tokyo.',
 'currency': '\n\nThe currency of Japan is the Japanese Yen (¥).',
 'places': " Here are some amazing places to visit in Tokyo:\n\n1. Tokyo Skytree - the tallest tower in Japan with observation decks offering stunning views of the city.\n\n2. Shibuya Crossing - the famous intersection known for its busy crowds and neon lights, featured in many movies and TV shows.\n\n3. Meiji Shrine - a peaceful Shinto shrine surrounded by a beautiful forest in the heart of the city.\n\n4. Tsukiji Fish Market - the largest fish market in the world, where you can witness the famous tuna auctions and enjoy fresh seafood.\n\n5. Akihabara - the center of Japan's otaku culture, known for its electronic shops, anime and manga stores, and maid cafes.\n\n6. Ueno Park - a beautiful park with museums, temples, shrines, and a zoo, perfect for a relaxing day out.\n\n7. Ginza - a high-end shopping district with luxury boutiques, department store